In [1]:
import os
import click
import re
import json
import tempfile
import torch
import dnnlib

from training import training_loop
from metrics import metric_main
from torch_utils import training_stats
from torch_utils import custom_ops

In [2]:
# =======================================================
# Stylegan2 configuration
# =======================================================
#    python train.py --outdir=./training-runs \
#                    --data=./datasets/ffhq.zip \
#                    --gpus=8 \
#                    --cfg=stylegan2 \
#                    --mirror=1 \
#                    --aug=noaug
args = dnnlib.EasyDict()
args.num_gpus = 8
args.image_snapshot_ticks = 50
args.network_snapshot_ticks = 50
args.metrics = ['fid50k_full']
args.random_seed = 0
args.training_set_kwargs = dnnlib.EasyDict(class_name='training.dataset.ImageFolderDataset', 
                                           path='./datasets/ffhq.zip', 
                                           use_labels=True, 
                                           max_size=None, 
                                           xflip=False)
args.data_loader_kwargs = dnnlib.EasyDict(pin_memory=True, 
                                          num_workers=3, 
                                          prefetch_factor=2)
args.training_set_kwargs.resolution  = 1024
desc = 'ffhq' # training_set.name
args.training_set_kwargs.use_labels = True
args.training_set_kwargs.max_size = 70000
args.training_set_kwargs.use_labels = False # -cond False
args.training_set_kwargs.random_seed = args.random_seed
desc += '-mirror' # # mirrory=1
args.training_set_kwargs.xflip = True  # mirrory=1
cfg = 'stylegan2'
desc += f'-{cfg}'
cfg_specs = {
    'auto':      dict(ref_gpus=-1, kimg=25000,  mb=-1, mbstd=-1, fmaps=-1,  lrate=-1,     gamma=-1,   ema=-1,  ramp=0.05, map=2), # Populated dynamically based on resolution and GPU count.
    'stylegan2': dict(ref_gpus=8,  kimg=25000,  mb=32, mbstd=4,  fmaps=1,   lrate=0.002,  gamma=10,   ema=10,  ramp=None, map=8), # Uses mixed-precision, unlike the original StyleGAN2.
    'paper256':  dict(ref_gpus=8,  kimg=25000,  mb=64, mbstd=8,  fmaps=0.5, lrate=0.0025, gamma=1,    ema=20,  ramp=None, map=8),
    'paper512':  dict(ref_gpus=8,  kimg=25000,  mb=64, mbstd=8,  fmaps=1,   lrate=0.0025, gamma=0.5,  ema=20,  ramp=None, map=8),
    'paper1024': dict(ref_gpus=8,  kimg=25000,  mb=32, mbstd=4,  fmaps=1,   lrate=0.002,  gamma=2,    ema=10,  ramp=None, map=8),
    'cifar':     dict(ref_gpus=2,  kimg=100000, mb=64, mbstd=32, fmaps=1,   lrate=0.0025, gamma=0.01, ema=500, ramp=0.05, map=2),
}
spec = dnnlib.EasyDict(cfg_specs[cfg])
args.G_kwargs = dnnlib.EasyDict(class_name='training.networks.Generator', z_dim=512, w_dim=512, mapping_kwargs=dnnlib.EasyDict(), synthesis_kwargs=dnnlib.EasyDict())
args.D_kwargs = dnnlib.EasyDict(class_name='training.networks.Discriminator', block_kwargs=dnnlib.EasyDict(), mapping_kwargs=dnnlib.EasyDict(), epilogue_kwargs=dnnlib.EasyDict())
args.G_kwargs.synthesis_kwargs.channel_base = args.D_kwargs.channel_base = int(spec.fmaps * 32768)
args.G_kwargs.synthesis_kwargs.channel_max = args.D_kwargs.channel_max = 512
args.G_kwargs.mapping_kwargs.num_layers = spec.map
args.G_kwargs.synthesis_kwargs.num_fp16_res = args.D_kwargs.num_fp16_res = 4 # enable mixed-precision training
args.G_kwargs.synthesis_kwargs.conv_clamp = args.D_kwargs.conv_clamp = 256 # clamp activations to avoid float16 overflow
args.D_kwargs.epilogue_kwargs.mbstd_group_size = spec.mbstd

args.G_opt_kwargs = dnnlib.EasyDict(class_name='torch.optim.Adam', lr=spec.lrate, betas=[0,0.99], eps=1e-8)
args.D_opt_kwargs = dnnlib.EasyDict(class_name='torch.optim.Adam', lr=spec.lrate, betas=[0,0.99], eps=1e-8)
args.loss_kwargs = dnnlib.EasyDict(class_name='training.loss.StyleGAN2Loss', r1_gamma=spec.gamma)


args.total_kimg = spec.kimg
args.batch_size = spec.mb
args.batch_gpu = spec.mb // spec.ref_gpus
args.ema_kimg = spec.ema
args.ema_rampup = spec.ramp

In [3]:
import pprint
pprint.pprint(args)

{'D_kwargs': {'block_kwargs': {},
              'channel_base': 32768,
              'channel_max': 512,
              'class_name': 'training.networks.Discriminator',
              'conv_clamp': 256,
              'epilogue_kwargs': {'mbstd_group_size': 4},
              'mapping_kwargs': {},
              'num_fp16_res': 4},
 'D_opt_kwargs': {'betas': [0, 0.99],
                  'class_name': 'torch.optim.Adam',
                  'eps': 1e-08,
                  'lr': 0.002},
 'G_kwargs': {'class_name': 'training.networks.Generator',
              'mapping_kwargs': {'num_layers': 8},
              'synthesis_kwargs': {'channel_base': 32768,
                                   'channel_max': 512,
                                   'conv_clamp': 256,
                                   'num_fp16_res': 4},
              'w_dim': 512,
              'z_dim': 512},
 'G_opt_kwargs': {'betas': [0, 0.99],
                  'class_name': 'torch.optim.Adam',
                  'eps': 1e-08,
      

In [4]:
desc

'ffhq-mirror-stylegan2'

In [47]:
device = torch.device('cuda:0')
print(device)

cuda:0


In [46]:
rank = 1
map_location = {'cuda:%d' % 0: 'cuda:%d' % rank}
map_location

{'cuda:0': 'cuda:1'}

In [48]:
device = torch.device('cpu')
print(device)

cpu
